# Unsupervised learning techniques

Unsupervised learning is a machine learning technique used to identify patterns and structures in data without using labeled training examples. The algorithm explores the data to find hidden patterns, groupings, or relationships, making it useful for tasks such as clustering, dimensionality reduction
(which we have explored in the previous chapter), and anomaly detection.

## Clustering

Clustering is a technique consisting of grouping similaring instances into clusters, it is throughly used in data analysis, image segmentation, customer
recommendations ...etc. Just like in classification, the algorithm separate the data in classes however here the data is not labeled. We are going to talk
about the 2 most used clustering methods: k-means and dbscan.

### K-means

K-Means clustering is one of the most popular and straightforward unsupervised learning algorithms used for partitioning a dataset into a set of distinct 
groups, or clusters. The primary goal of K-Means is to divide the data into 𝑘 clusters, where each data point belongs to the cluster with the nearest 
mean, serving as a prototype of the cluster.

In [4]:
from sklearn.datasets import make_blobs
import numpy as np
from sklearn.cluster import KMeans

blob_centers = np.array([[ 0.2,  2.3], [-1.5 ,  2.3], [-2.8,  1.8], [-2.8,  2.8], [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers, cluster_std=blob_std, random_state=7)
# The code above just generates a randomly clustered dataset
k = 5
kmeans = KMeans(n_clusters=k, random_state=42) # We have to specify the number of clusters the algorithm needs to find 
y_pred = kmeans.fit_predict(X)
print(y_pred)
# The algorithm store the centroid of the clusters he have found too
print(kmeans.cluster_centers_)
# The algorithm assign all new instance to the centroid it is the closest to.
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
print(kmeans.predict(X_new))

[2 2 4 ... 1 4 2]
[[-0.066884    2.10378803]
 [-2.79290307  2.79641063]
 [-2.80214068  1.55162671]
 [-1.47468607  2.28399066]
 [ 0.47042841  2.41380533]]
[0 4 1 1]


Although this method will be accurate and most instances will be labeled correctly, some of them that are far from the centroids will probably be 
mislabeled. So it might be useful to not simply assign an instance to a cluster(which is called _hard cluster_) but to get a score for each instance about its probably of belonging in a particular cluster. This score can be calculated using the distance of the instance relative to the centroids of clusters or 
a similarity function such as the _Gaussian Radial Basis_ function(discussed in _end to end project_).

In [5]:
kmeans.transform(X_new).round(2)

array([[0.12, 2.9 , 2.84, 1.5 , 0.63],
       [3.07, 5.85, 5.82, 4.48, 2.56],
       [3.07, 0.29, 1.46, 1.69, 3.52],
       [2.96, 0.36, 0.97, 1.54, 3.47]])

One way to implement the algorithm is to randomly pick _k_ instances as centroids and start labeling the data, while doing that we also keep updating the
centroid until they stop moving. This solution is guaranteed to find a satisfying result in a relatively small number of steps, the problem is that it is
likely to stop at a local optimum instead of the global one. Another method more accurate than setting the centroid at random initially is setting them up
approximately where we think they should be(demonstrated in the following code):

In [6]:
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1, random_state=42)
kmeans.fit(X)

KMeans(init=array([[-3,  3],
       [-3,  2],
       [-3,  1],
       [-1,  2],
       [ 0,  2]]),
       n_clusters=5, n_init=1, random_state=42)

Another solution is to also run the algorithm with random centroids multiple times and keep the best final result(by setting the _n\_init_ parameter to
the number of times we want to run the algorithm). The measure of how well the algorithm is performing is called _inertia_. Inertia is the sum of the
squared distance between the instances and their closest centroids(so the best model is the model with the lowest inertia).

In [7]:
print(kmeans.inertia_)# The score attribute of our model will return the negative of the inertia.

211.5985372581684


An amelioration of the kmeans algorithm for choosing the centroids is called _kmeans++_. In standard K-Means, the initial centroids are selected randomly 
from the data points. This random selection can sometimes lead to poor clustering results or slow convergence, particularly if the initial centroids are 
not well-placed relative to the true cluster centers. Poor initialization can cause the algorithm to converge to a local minimum, resulting in suboptimal 
clusters. K-Means++ addresses this issue by carefully choosing the initial centroids in a way that spreads them out more evenly across the data, increasing 
the chances of finding good clusters. The algorithm works by choosing randomly a first centroid, after that for each instance the algorithm calculated
its probability of being selected as a centroid, the probability being computed is the following:
$$\frac{D(x^{(i)})^2}{\sum_{j=1}^{m}D(x^{(j)})^2} $$
Where $D(x^{(i)})$ is the distance between the instance and its closest centroid. Finally this steps are repeated until we have _k_ centroids.

__Side Note__: It is possible to use the _MiniBatchMeans()_ class when the dataset does not fit entirely in the memory.

Another problem of clustering algorithms is choosing the right number of clusters. One of the most precise way to do that is to use the silhouette score
of each instance. The __Silhouette Score__ measures how similar a data point is to its own cluster (cohesion) compared to other clusters (separation). It 
provides an indication of how well-separated the clusters are and how tightly grouped the data points within each cluster are. It is calculated with the
following formula:
$$\frac{(b - a)}{max(b, a)} $$
Where _a_ is the mean distance between the given instance and the other instances inside the same cluster and _b_ is the average distance between the data 
point and all the points in the nearest neighboring cluster. Note that this score vary from 1 to -1, an instance with a score close to 1 is well inside its
cluster and far from the other clusters, an instance with a score close to 0 is close to a boundary between multiple clusters and an instance close to -1
might have not been assigned properly.

In [8]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

np.float64(0.655517642572828)

__Important Note__: It is essential to scale the input features before performing kmeans it might not guarantee perfection but it greatly help in enhancing
precision.

Now we are going to look at a few practical examples of kmeans clustering usage.

#### Image Segmentation

We are going to implement a simple color segmentation algorithm using kmeans clustering and the _PIL_ library.

In [9]:
import PIL
import PIL.Image

image = np.asarray(PIL.Image.open('./data/flower.jpeg'))
print(image.shape)
# Then we can reshape the array into an array of RGB color that we are going to generate clusters from.
X = image.reshape(-1, 3)
kmeans = KMeans(n_clusters=8, random_state=42).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

(1313, 736, 3)


#### KMeans clustering for semi supervised learning

Let's test this using the MNIST handwritten digits datasets:

In [15]:
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
import numpy as np

X_digits, y_digits = load_digits(return_X_y=True)
X_train, y_train = X_digits[:1400], y_digits[:1400]
X_test, y_test = X_digits[1400:], y_digits[1400:]

# Let's use the labels of 50 instances (of course all the instances are labeled but for learning purpose we will pretend only 50 of them are labeled)
n_labeled = 50
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])
# Let's print the accuracy
print("Accuracy score before clustering")
print(log_reg.score(X_test, y_test))

# Now we can train a cluster with these 50 centroids
kmeans = KMeans(n_clusters=n_labeled, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

# Assigning the proper labels for our 50 instances
y_representative_digits = np.array([
    1, 3, 6, 0, 7, 9, 2, 4, 8, 9,
    5, 4, 7, 1, 2, 6, 1, 2, 5, 1,
    4, 1, 3, 3, 8, 8, 2, 5, 6, 9,
    1, 4, 0, 6, 8, 3, 4, 6, 7, 2,
    4, 1, 0, 7, 5, 1, 9, 9, 3, 7
])

# We retrain the model to work with the 50 defined centroids
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_representative_digits, y_representative_digits)
print("Accuracy score after clustering")
print(log_reg.score(X_test, y_test))


Accuracy score before clustering
0.7581863979848866
Accuracy score after clustering
0.16624685138539042


## DBSCAN (Density Based Spatial Clustering of Applications with Noise)

The dbscan algorithm define clusters as continous region of high density. The algorithm at first finds how many instances are at the distance $\varepsilon$ from a random instance(this region is called the $\varepsilon$-neighborhood). If an instance has at least _min\_instances_ in its neighborhood then it is 
considered a core instance. In conclusion a cluster will be defined by a long sequence of core instance(any instance that is not a _core instance_ and is 
not neighbor to one is considered an _anomaly_). Let's test it on the moon dataset.

In [16]:
from sklearn.datasets import make_moons
from sklearn.cluster import DBSCAN

X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X, y)

DBSCAN(eps=0.05)

## Gaussian mixture models

Gaussian Mixture Models (GMMs) are a type of probabilistic model used in to represent a mixture of multiple Gaussian distributions. GMMs are particularly 
useful for clustering, density estimation, and anomaly detection. Unlike K-Means, which assigns each data point to a single cluster, GMMs allow for soft 
clustering, where each data point can belong to multiple clusters with different probabilities. There are several GMM variants. In the simplest variant, 
implemented in the _GaussianMixture_ class, you must know in advance the number _k_ of Gaussian distributions. The dataset X is assumed to have been 
generated through the a probabilistic process. For each instance a cluster is picked randomly among k clusters, the probability of choosing a cluster j is 
its weight $\phi$. If the $i^{th}$ instance is assigned to the $j^{th}$ cluster then the location of this instance is sampled randomly from the distribution
of mean $\mu$ and of covariance $\varSigma$. Given the dataset we first need to estimate the model's parameter and after that look to apply the algorithm.

In [18]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)
# Let's look at the models estimated parameters
print(f"Estimated weight:\n {gm.weights_}\n")
print(f"Estimated mean:\n {gm.means_}\n")
print(f"Estimated covariance:\n {gm.covariances_}\n")
# Now we can use the model to try to predict the belonging of a new instance by calling the predict method(hard clustering) or predict_proba(for soft clustering)

Estimated weight:
 [0.58170308 0.21375196 0.20454497]

Estimated mean:
 [[ 0.49323201  0.26069758]
 [ 1.71726611 -0.07880263]
 [-0.74487174  0.56482113]]

Estimated covariance:
 [[[ 0.16279376 -0.09345901]
  [-0.09345901  0.28709004]]

 [[ 0.06572952  0.06881017]
  [ 0.06881017  0.08834991]]

 [[ 0.05790923  0.06413371]
  [ 0.06413371  0.08734471]]]



Let's use Gaussian Mixture for anomaly detection.

### Gaussian Mixture for anomaly detection

Gaussian mixture classify all the instances that are located in a low density region as _anomaly_. The threshold of the density needs to be specified
manually.

In [20]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 2)
anomalies = X[densities < density_threshold]

Another problem, similar to the one in kmeans clustering, is finding the right number of cluster to pass to the algorithm. As we cannot use our usual
metrics such as the silhouette score, we can find the model that best minimize the Bayesian Information Criterion:
$$BIC = \log(m)p - 2\log(\hat{\mathscr{L}})$$
Or the Aikake information criterion:
$$AIC = 2p - 2\log(\hat{\mathscr{L}})$$
m being the number of instances, p the number of parameters and $\hat{\mathscr{L}}$ is the maximized value of the likelihood function model. Note
that computing all of this is doable by simply calling the _bic()_ and the _aic()_ method

In [22]:
print(f"Number of clusters found by BIC: {gm.bic(X)}")
print(f"Number of clusters found by AIC: {gm.aic(X)}")

Number of clusters found by BIC: 2801.1544266441592
Number of clusters found by AIC: 2717.722586901463
